In [1]:
# change the cwd
import os
os.chdir(os.getcwd()+"/HLP_Project/")

In [2]:
# use cython compiled modules for faster execution
from libs.cython.pyx.edgelistParser import edgelistParser
from libs.cython.pyx.utils import list2matrix, getInOutDegree
from libs.cython.cpp.pageRankCpp import cppLocalPageRank as localPageRank
from libs.cython.cpp.pageRankCpp import cppLocalPageRank_weight as localPageRank_weight

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import datetime

In [4]:
_ = """
Dataset Types
../datasets/facebook/*.edges -> raw_list
../datasets/facebook_combined.txt -> list
../datasets/p2p-Gnutella31.txt -> list
../datasets/Wiki-Vote.txt -> list
../datasets/CA-HepPh.txt -> raw_list
../datasets/com-dblp.ungraph.txt -> list
../datasets/Email-Enron.txt -> raw_list
../datasets/Email-EuAll.txt -> list
"""

In [5]:
dataset = "../datasets/facebook/0.edges"
list_type = "raw_list"

In [6]:
# load a dataset, in both weighted and unweighted forms
A_weighted, N, original_idxs = edgelistParser(dataset, list_type, True)
A, _, _ = edgelistParser(dataset, list_type, False)

In [7]:
inDeg, _ = getInOutDegree(A_weighted)
print("Nodes:", N, "\b, AvgDeg:", np.array(inDeg).mean())

Nodes: 333, AvgDeg: 30.25825825825826


In [8]:
print("Unweighted:")
%timeit localPageRank(A, c=0.85, return_only_neighbours=True, epsilon=1e-5, max_iters=50)
print("Weigted:")
%timeit localPageRank_weight(A_weighted, c=0.85, return_only_neighbours=True, epsilon=1e-5, max_iters=50)

Unweighted:
25.6 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Weigted:
23.9 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
_ = """
Nodes: 333, AvgDeg: 30.25825825825826
    Unweighted:
    25.6 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
    Weigted:
    23.9 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)



"""